In [1]:
import glob
import os
import logging
import sys

import numpy as np
import tifffile
from skimage import io
from cellpose import models
import napari
import torch

In [2]:
r = logging.getLogger()
r.setLevel(logging.INFO)
h = logging.StreamHandler(sys.stdout)
h.setLevel(logging.INFO)
r.addHandler(h)

In [3]:
viewer = napari.Viewer()

No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'
VERSION = 2.3.0

The bioformats_package.jar is not present.Can only use Python backend for reading/writing images.
Resource 'XMLSchema.xsd' is already loaded


In [4]:
images = sorted(glob.glob("Data/Training/sample_images/*.tif"))
len(images)

23

In [24]:
d = torch.device('mps')
# change gpu=True if on windows, and get rid of device
# model = models.Cellpose(gpu=True, model_type='cyto2')
model = models.Cellpose(gpu=False, device=d, model_type='cyto2')

In [59]:
x = tifffile.imread(images[15])
x.shape

(2044, 2048)

Add the image to the viewer. What do you notice about the image?

In [72]:
viewer.layers.clear()
viewer.add_image(x)

<Image layer 'x' at 0x4f2bdbfa0>

Try the cellpose cyto2 model

In [73]:

masks, flows, _, _ = model.eval(x, channels=[0, 0], diameter=40)

In [74]:
viewer.add_labels(masks)

<Labels layer 'masks' at 0x2ffd99000>

### Label images for training
- View image
- model.eval the image
- add labels
- edit the labels
- get labels out of the viewer
- create a stack with the image and the mask
- save to file

In [5]:
files = sorted(glob.glob("Data/Training/training_images/*.tif"))
len(files)

23

In [30]:
images = list()
masks = list()

for f in files:
    x = tifffile.imread(f)
    images.append(x[0])
    masks.append(x[1].astype(np.uint16))
    

In [7]:
### Training


In [9]:
d = torch.device('mps')
model = models.CellposeModel(gpu=False, device=d, model_type='cyto2')

>> cyto2 << model set to be used
see https://pytorch.org/docs/stable/backends.html?highlight=mkl
>>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)


In [31]:
model.train(images, masks, channels=[0, 0], save_path='models', n_epochs=400,
            nimg_per_epoch=24, model_name='custom', batch_size=32,
            min_train_masks=1)

computing flows for labels


100%|██████████| 23/23 [01:44<00:00,  4.53s/it]


>>>> median diameter set to = 30
>>>> mean of training label mask diameters (saved to model) 124.823
>>>> training network with 2 channel input <<<<
>>>> LR: 0.20000, batch_size: 32, weight_decay: 0.00001
>>>> ntrain = 23
>>>> nimg_per_epoch = 24
Epoch 0, Time  1.2s, Loss 0.1368, LR 0.0000
saving network parameters to models/models/custom
Epoch 5, Time  6.7s, Loss 0.1427, LR 0.1111
Epoch 10, Time 12.1s, Loss 0.1522, LR 0.2000
Epoch 20, Time 22.8s, Loss 0.1412, LR 0.2000
Epoch 30, Time 33.6s, Loss 0.1304, LR 0.2000
Epoch 40, Time 44.2s, Loss 0.1381, LR 0.2000
Epoch 50, Time 54.8s, Loss 0.1446, LR 0.2000
Epoch 60, Time 65.5s, Loss 0.1440, LR 0.2000
Epoch 70, Time 76.0s, Loss 0.1269, LR 0.2000
Epoch 80, Time 86.9s, Loss 0.1439, LR 0.2000
Epoch 90, Time 97.5s, Loss 0.1373, LR 0.2000
Epoch 100, Time 107.9s, Loss 0.1378, LR 0.2000
saving network parameters to models/models/custom
Epoch 110, Time 118.5s, Loss 0.1356, LR 0.2000
Epoch 120, Time 128.9s, Loss 0.1299, LR 0.2000
Epoch 130, Time 139

KeyboardInterrupt: 

Now check how well the model is doing.

This model only has 3 return values, so get rid of the last 

In [26]:
x = tifffile.imread(files[3])[0]
masks, flows, _ = model.eval(x, channels=[0, 0])

In [27]:
viewer.layers.clear()
viewer.add_image(x)
viewer.add_labels(masks)

<Labels layer 'masks' at 0x2e21d84c0>

In [28]:
viewer.add_image(flows[2])

<Image layer 'Image' at 0x30059a800>

In [22]:
x.min(),

(-3.12733, 3.7140808, 0.2506582)